## Entraînement du modèle DIST_REL_C_01

Nous allons ici entraîner un modèle prenant les mêmes entrées et devant prédire les mêmes distances que le modèle DIST_REL_C_RNN_01 (voir notebook 9.2).

La différence est que nous allons utiliser un modèle prenant des entrées "plates" et qui ne sera pas un RNN. Notre modèle va avoir une topologie très similaire aux différents modèles DELTA_DIST+H.

L'objectif ici est de chercher à savoir si l'utilisation des RNN apporte quelque chose ou pas. Nous allons donc entraîner un réseau profond entièrement connecté et comparer les résultats des deux modèles.

#### Chemin des fichiers

In [8]:
minimal_prepared_input_loc = "../data/DIST_REL_C_RNN/minimal_prepared_input.h5"
minimal_labels_loc = "../data/DIST_REL_C_RNN/minimal_labels.h5"

train_prepared_input_loc = "../data/DIST_REL_C_RNN/train_set_prepared_input.h5"
train_labels_loc = "../data/DIST_REL_C_RNN/train_set_labels.h5"

models_loc = "../models/DIST_REL_C_01/11.0/"
logs_loc = "../models/DIST_REL_C_01/11.0/"

### Définition des fonctions de coût et de validation

#### RMSE (coût)

In [9]:
def rmse(pred, targets):
    with tf.name_scope("rmse_loss"):
        return tf.sqrt(tf.reduce_mean(tf.squared_difference(pred, targets)), name="rmse")

#### Fonction d'évaluation des performances (opposé du RMSE)


In [10]:
def rmse_valid(pred, targets, inputs):
    with tf.name_scope("rmse_validation"):
        return -rmse(pred, targets)

## Création du RN

In [11]:
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.optimizers import Adam
from tflearn.data_preprocessing import DataPreprocessing
import math


def creer_RN(epsilon=1e-8, learning_rate=0.001, dropout_val=0.99, stddev_init=0.001,
             hidden_act='relu', outlayer_act='prelu', weight_decay=0.001, width=870, depth=3,
             validation_fun=rmse_valid, cost_fun=rmse):

    # On créé l'initialisateur de tenseur avec une loi normale tronquée. sigma = stddev_init, et les 
    # valeurs à plus de 2sigma sont re-tirées
    winit = tfl.initializations.truncated_normal(stddev=stddev_init, dtype=tf.float32, seed=None)
    
    # On créé l'input du RN
    network = input_data(shape=[None, 870], name='input')
    
    # On créé les couches cachées
    for i in range(depth):
        network = fully_connected(network, width, activation=hidden_act, name='fc'+str(i), weights_init=winit,
                                  weight_decay=weight_decay)
        # On détruit des neurones aléatoirement avec une la probabilité donnée en entrée
        network = dropout(network, dropout_val)
    
    # On ajoute la couche de sortie du réseau
    # Fonction d'activation prelu
    # Initilisée avec la loi normale tronquée
    network = fully_connected(network, 1, activation=outlayer_act, name='outlayer', weights_init=winit)
    
    adam = Adam(learning_rate=learning_rate, epsilon=epsilon)
    
    # Couche d'évaluation du modèle. Utilisation d'une descente stochastique Adam
    # Learning rate = 0.05
    # Loss = fonction définie rmse
    network = regression(network, optimizer=adam,
    loss=cost_fun, metric=validation_fun, name='target')
            
    return network

## Entraînement du RN

In [5]:
import h5py
import tflearn as tfl
import time
from scipy import sparse
import numpy as np
import gc
import tensorflow as tf


def train_model(input_X, labels_y, model_name, model_path, logs_path, samples_per_batch=1000, epochs=5,
                learning_rate=0.001, epsilon=1e-8, dropout=0.99, stddev_init=0.001, hidden_act='relu',
                outlayer_act='prelu', cost_fun=rmse, validation_fun=rmse_valid, width=870, depth=2):
    
    total_start_time = time.time()

    tf.reset_default_graph()
    
    # On créé le réseau 
    network = creer_RN(learning_rate=learning_rate, epsilon=epsilon, dropout_val=dropout,
                       stddev_init=stddev_init, hidden_act=hidden_act, outlayer_act=outlayer_act,
                       validation_fun=validation_fun, cost_fun=cost_fun, width=width, depth=depth)

    # On créé le modèle
    model = tfl.DNN(network, tensorboard_verbose=3, tensorboard_dir=logs_path)

    # Entraînement
    model.fit(X_inputs=input_X,Y_targets=labels_y, batch_size=samples_per_batch,
              shuffle = True, snapshot_step=100, validation_set=0.1,
              show_metric=True, run_id=model_name, n_epoch=epochs)

    # Sauvegarde du modèle
    model.save(model_path + model_name + ".tflearn")
